In [2]:
import pandas as pd
import csv
import os

setting up the data 

In [ ]:
# setting up the data
file_names = os.listdir('data/raw data')

['2018-frenchopen-matches-mixed.csv',
 '2023-wimbledon-matches-mixed.csv',
 '2019-frenchopen-points-doubles.csv',
 '2017-ausopen-points.csv',
 '2018-wimbledon-points.csv',
 '2023-wimbledon-matches-doubles.csv',
 '2020-frenchopen-matches-doubles.csv',
 '2013-frenchopen-points.csv',
 '2024-usopen-points-doubles.csv',
 '2014-wimbledon-points.csv',
 '2021-frenchopen-points-mixed.csv',
 '2018-ausopen-matches.csv',
 '2015-frenchopen-points.csv',
 '2011-frenchopen-points.csv',
 '2016-ausopen-matches.csv',
 '2023-wimbledon-matches.csv',
 '2020-usopen-matches-doubles.csv',
 '2022-usopen-points.csv',
 '2019-ausopen-matches-doubles.csv',
 '2018-frenchopen-matches-doubles.csv',
 '2019-ausopen-points.csv',
 '2019-wimbledon-matches-mixed.csv',
 '2016-wimbledon-points.csv',
 '2020-ausopen-points-mixed.csv',
 '2022-usopen-matches.csv',
 '2021-ausopen-points.csv',
 '2012-wimbledon-matches.csv',
 '2012-frenchopen-points.csv',
 '2021-frenchopen-points-doubles.csv',
 '2014-usopen-matches.csv',
 '2019-fren